Copyright (C) 2021 TU Dresden, Center for Information Services and
High Performance Computing

Copyright (C) 2025 Markus Schmidl

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import pickle
import seaborn as sns
import pandas as pd
from experiment_utils import Experiment, ExperimentFilter, Plotting
from experiment_utils.metricq import Counter

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('metricq-power-distribution'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
POWER = 'Power in bins of 0.1W'
COUNT = 'Number of samples'
PERCENT = 'Number of samples [%]'

IDLE_THRESHOLD = 200
MAX_POWER_THRESHOLD = 900

In [ ]:
with open(experiment.path / 'counters.pickle', 'rb') as f:
    counters = pickle.load(f)

    total_counter = Counter()
    for hostname, counter in counters.items():
        for key, value in counter.items():
            total_counter[key] += value

df = pd.DataFrame.from_dict(total_counter, orient='index')
df.reset_index(inplace=True)
df.columns = [POWER, COUNT]

total_count = sum(df[COUNT].to_list())
print(f'Total number of samples {total_count}')
df[PERCENT] = df[COUNT] / total_count * 100

below = sum(df[df[POWER] < IDLE_THRESHOLD][COUNT].to_list()) / total_count
above = sum(df[df[POWER] >= IDLE_THRESHOLD][COUNT].to_list()) / total_count
print(f"{below*100}% below, {above*100}% above IDLE_THRESHOLD of {IDLE_THRESHOLD}W")

df

In [ ]:
ax = sns.lineplot(x=POWER, y=PERCENT, data=df)
ax.set_xlim(0, MAX_POWER_THRESHOLD)
ax.set_ylim(ymin=0)

Plotting.savefig(experiment, f'power_histogram.pdf', annotations_x_offset=0.05, annotations_y_offset=0.01, annotations_y_spacing=0.025)

In [ ]:
ax = sns.ecdfplot(x=POWER, weights=COUNT, stat='percent', data=df)
ax.set_xlim(0, MAX_POWER_THRESHOLD)
ax.set_ylim(0, 100)
Plotting.savefig(experiment, f'power_ecdf.pdf', annotations_x_offset=0.05, annotations_y_offset=0.01, annotations_y_spacing=0.025)